In [2]:
cd ..

c:\Users\14102\Brown\Realizing_Rights\RealizingRights


In [3]:
from utilities.web_utils import recurse_scan_all_unique_links_in_site, closest_link_match, make_https, prepend_root_to_url, make_driver_utils
from url_scraper.link_data import LinkData
from objects.scrape_lists import blacklist_terms, link_keywords, board_meeting_keywords, social_media_sites

import pandas as pd

In [4]:
district_df = pd.read_excel(io='data/USSchoolDistrictWebsiteInfo.xlsx', sheet_name='ELSI Export', header=6)
district_df['URL'] = district_df['URL'].apply(lambda x: make_https(url=x))

district_df['NumStudents'] = pd.to_numeric(district_df['NumStudents'], errors='coerce')
district_df_with_urls = district_df[~district_df['URL'].isna()]
district_df_with_urls['URL'] = district_df_with_urls['URL'].apply(lambda x: prepend_root_to_url(base_url=x.lower(), prefix=''))
district_df_with_urls = district_df_with_urls[district_df_with_urls['NumStudents'] > 0]
district_df_with_urls = district_df_with_urls[district_df_with_urls['NumSchools'] > 0].reset_index(drop=True)
district_df_with_urls.sort_values(by='NumStudents', ascending=False, inplace=True)

c:\Users\14102\anaconda3\envs\real_right_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [8]:
drvr, actions, wait = make_driver_utils()

url_data = {}
for _, row in district_df_with_urls[:100].iterrows():

    url = row['URL']
    start_link_set = set()
    dist_info = []
    start_link_set.add(LinkData(link_text='BASE', link_url=url, depth=0))
    recursed_lcl_links, recursed_ext_links = set(), set()
    recursed_lcl_links, recursed_ext_links = recurse_scan_all_unique_links_in_site(url=url,
                                                                                    base_url=url, 
                                                                                    local_link_set=start_link_set,
                                                                                    external_link_set=set(),
                                                                                    drvr=drvr, 
                                                                                    actions=actions,
                                                                                    blacklist_terms=blacklist_terms,
                                                                                    wait=wait,
                                                                                    link_keywords=link_keywords)

    cur_sim = 60
    best_link = None
    lcl_links_sorted = sorted(list(recursed_lcl_links), key=lambda x: x.depth_found)
    ext_links_sorted = sorted(list(recursed_ext_links), key=lambda x: x.depth_found)
    for lcl_link in lcl_links_sorted:
        if cur_sim == 100:
            break
        sim = closest_link_match(lcl_link.link_text, board_meeting_keywords)
        if sim > cur_sim:
            best_link = lcl_link
            cur_sim = sim

    for ext_link in ext_links_sorted:
        if cur_sim == 100:
            break
        if ext_link.link_text:
            sim = closest_link_match(ext_link.link_text, board_meeting_keywords)
            if sim > cur_sim:
                best_link = ext_link
                cur_sim = sim

    # Identify External Links Pointing to social media sites 
    sites_identified = {}
    num_ext_links = len(ext_links_sorted)
    ext_id = 0
    social_media_sites = ['youtube', 'vimeo', 'facebook', 'twitter']
    while ext_id < num_ext_links:
        ext_link = ext_links_sorted[ext_id]
        scl_id = 0
        while scl_id < len(social_media_sites):
            if social_media_sites[scl_id] in ext_link.link_url:
                sites_identified[social_media_sites.pop(scl_id)] = ext_link.depth_found
            scl_id += 1

        ext_id += 1

        if len(social_media_sites) == 0:
            break
    
    if best_link:
        url_data[row['Agency ID']] = (sites_identified, best_link.link_text, best_link.link_url, best_link.depth_found)
    else: 
        url_data[row['Agency ID']] = (sites_identified, None, None, None)
    print(sites_identified, best_link)



{'youtube': 0, 'facebook': 0, 'twitter': 0, 'vimeo': 2} Video: https://lausd.wistia.com/medias/mkguffmxty
{} None
{} None
{'twitter': 0, 'facebook': 0} None
{'twitter': 0, 'youtube': 0, 'facebook': 0} Board Meetings & Workshops: https://www.browardschools.com/boardmeetings
{'twitter': 0, 'facebook': 0, 'youtube': 0} Minutes: https://www.hillsboroughschools.org/domain/5433
{} None
{'youtube': 0, 'twitter': 0, 'facebook': 0} Board: https://www.houstonisd.org/domain/40263
{'twitter': 0, 'youtube': 0, 'facebook': 0} Board Meetings: https://www.palmbeachschools.org/domain/255
{'facebook': 0, 'twitter': 0, 'youtube': 0} School Board Meetings: https://www.fcps.edu/school-board/school-board-meetings


In [ ]:
df = pd.DataFrame(columns=['District ID', 'brd_mtn_URL_text', 'brd_mtn_URL_Link', 'brd_mtn_URL_depth', 'facebook', 'youtube', 'twitter', 'vimeo'])


for url, data in url_data.items():
    row = {
        'District ID': url,
        'brd_mtn_URL_text': data[1],
        'brd_mtn_URL_Link': data[2],
        'brd_mtn_URL_depth': data[3],
        'facebook': None,
        'youtube': None,
        'twitter': None,
        'vimeo': None
    }

    if data[0]:
        row.update(data[0])

    df = df.append(row, ignore_index=True)

df

In [ ]:
scanned_dist_df = df.merge(district_df_with_urls, left_on='District ID', right_on='Agency ID')
scanned_dist_df.to_csv('data/SampleOutput.csv')

In [1]:
district_df = pd.read_excel(io='data/USSchoolDistrictWebsiteInfo.xlsx', sheet_name='ELSI Export', header=6)
district_df['NumStudents'] = pd.to_numeric(district_df['NumStudents'], errors='coerce')
district_df.sort_values(by='NumStudents', ascending=False).head(50)

NameError: name 'pd' is not defined

In [11]:
columns = ['District ID', 'brd_mtn_URL_text', 'brd_mtn_URL_Link', 'brd_mtn_URL_depth']

columns + social_media_sites

['District ID',
 'brd_mtn_URL_text',
 'brd_mtn_URL_Link',
 'brd_mtn_URL_depth',
 'youtube',
 'vimeo',
 'facebook',
 'twitter']